# Geospatial Data Analysis

This notebook demonstrates geospatial analysis and mapping.

**Libraries:**
- [Cartopy](https://scitools.org.uk/cartopy/) - Map projections and features
- [GeoPandas](https://geopandas.org/) - Geospatial data in DataFrames
- [Folium](https://python-visualization.github.io/folium/) - Interactive web maps
- [Shapely](https://shapely.readthedocs.io/) - Geometric objects

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Geospatial imports
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import geopandas as gpd
from shapely.geometry import Point, Polygon, LineString
import folium
from folium.plugins import HeatMap, MarkerCluster

%matplotlib inline

## Sample Data - World Cities

In [ ]:
cities = pd.DataFrame({
    "city": ["New York", "Los Angeles", "Chicago", "Houston", "Phoenix",
             "London", "Paris", "Tokyo", "Sydney", "Mumbai"],
    "lat": [40.7128, 34.0522, 41.8781, 29.7604, 33.4484,
            51.5074, 48.8566, 35.6762, -33.8688, 19.0760],
    "lon": [-74.0060, -118.2437, -87.6298, -95.3698, -112.0740,
            -0.1278, 2.3522, 139.6503, 151.2093, 72.8777],
    "population": [8.3, 3.9, 2.7, 2.3, 1.6, 8.9, 2.1, 13.9, 5.3, 20.4],
})
cities

## Cartopy - Map Projections

Cartopy provides different map projections for visualizing geographic data.

In [ ]:
fig = plt.figure(figsize=(15, 10))

# Plate Carree (standard lat/lon)
ax1 = fig.add_subplot(2, 2, 1, projection=ccrs.PlateCarree())
ax1.set_global()
ax1.add_feature(cfeature.LAND, facecolor="lightgray")
ax1.add_feature(cfeature.OCEAN, facecolor="lightblue")
ax1.add_feature(cfeature.COASTLINE, linewidth=0.5)
ax1.gridlines(draw_labels=True, linewidth=0.5, alpha=0.5)
ax1.set_title("Plate Carree Projection")

# Orthographic (globe view)
ax2 = fig.add_subplot(2, 2, 2, projection=ccrs.Orthographic(-80, 35))
ax2.set_global()
ax2.add_feature(cfeature.LAND, facecolor="lightgray")
ax2.add_feature(cfeature.OCEAN, facecolor="lightblue")
ax2.add_feature(cfeature.COASTLINE, linewidth=0.5)
ax2.set_title("Orthographic (Americas)")

# Mollweide (equal-area)
ax3 = fig.add_subplot(2, 2, 3, projection=ccrs.Mollweide())
ax3.set_global()
ax3.add_feature(cfeature.LAND, facecolor="lightgray")
ax3.add_feature(cfeature.OCEAN, facecolor="lightblue")
ax3.add_feature(cfeature.COASTLINE, linewidth=0.5)
ax3.set_title("Mollweide Projection")

# Robinson
ax4 = fig.add_subplot(2, 2, 4, projection=ccrs.Robinson())
ax4.set_global()
ax4.add_feature(cfeature.LAND, facecolor="lightgray")
ax4.add_feature(cfeature.OCEAN, facecolor="lightblue")
ax4.add_feature(cfeature.COASTLINE, linewidth=0.5)
ax4.set_title("Robinson Projection")

plt.tight_layout()
plt.show()

### Plotting Data on Maps

In [ ]:
fig = plt.figure(figsize=(14, 8))
ax = fig.add_subplot(1, 1, 1, projection=ccrs.Robinson())
ax.set_global()

ax.add_feature(cfeature.LAND, facecolor="#E0E0E0")
ax.add_feature(cfeature.OCEAN, facecolor="#B0D0E8")
ax.add_feature(cfeature.COASTLINE, linewidth=0.5)
ax.add_feature(cfeature.BORDERS, linestyle=":", linewidth=0.3, alpha=0.5)

# Plot cities
scatter = ax.scatter(
    cities["lon"], cities["lat"],
    c=cities["population"], s=cities["population"] * 20,
    cmap="YlOrRd", alpha=0.8, edgecolors="black", linewidth=0.5,
    transform=ccrs.PlateCarree(), zorder=5,
)

# Add city labels
for idx, row in cities.iterrows():
    ax.annotate(row["city"], xy=(row["lon"], row["lat"]),
                xytext=(5, 5), textcoords="offset points",
                fontsize=8, transform=ccrs.PlateCarree())

plt.colorbar(scatter, label="Population (millions)", shrink=0.5)
ax.set_title("Major World Cities by Population", fontsize=14)
plt.show()

## GeoPandas - Vector Data Operations

GeoPandas extends Pandas to support geospatial data.

In [ ]:
# Create GeoDataFrame from points
geometry = [Point(lon, lat) for lon, lat in zip(cities["lon"], cities["lat"])]
gdf_cities = gpd.GeoDataFrame(cities, geometry=geometry, crs="EPSG:4326")

print(f"CRS: {gdf_cities.crs}")
print(f"Geometry type: {gdf_cities.geometry.geom_type.unique()}")
gdf_cities

### Spatial Operations

In [ ]:
# Create LineString connecting cities
coords = list(zip(cities["lon"], cities["lat"]))
route = LineString(coords[:5])  # Connect first 5 cities
print(f"Route length (degrees): {route.length:.2f}")

# Centroid calculation
all_cities_union = gdf_cities.geometry.union_all()
centroid = all_cities_union.centroid
print(f"Centroid of all cities: ({centroid.x:.2f}, {centroid.y:.2f})")

# Distance calculation
ny = gdf_cities[gdf_cities["city"] == "New York"].geometry.iloc[0]
la = gdf_cities[gdf_cities["city"] == "Los Angeles"].geometry.iloc[0]
print(f"Distance NY to LA (degrees): {ny.distance(la):.2f}")

### GeoPandas Plot

In [ ]:
fig, ax = plt.subplots(figsize=(12, 8))
world = gpd.GeoDataFrame(
    geometry=[Polygon([(-180, -90), (180, -90), (180, 90), (-180, 90)])],
    crs="EPSG:4326"
)
world.plot(ax=ax, color="lightgray", edgecolor="white")
gdf_cities.plot(ax=ax, column="population", cmap="YlOrRd",
                markersize=gdf_cities["population"] * 10, legend=True)
ax.set_title("Cities GeoDataFrame Visualization")
ax.set_xlabel("Longitude")
ax.set_ylabel("Latitude")
plt.show()

## Folium - Interactive Web Maps

Folium creates interactive Leaflet.js maps.

In [ ]:
# Basic map with city markers
m = folium.Map(location=[40, -95], zoom_start=4, tiles="OpenStreetMap")

for idx, row in cities.iterrows():
    popup_text = f"{row['city']}<br>Population: {row['population']}M"
    folium.CircleMarker(
        location=[row["lat"], row["lon"]],
        radius=row["population"] * 2,
        popup=popup_text,
        color="red",
        fill=True,
        fill_color="red",
        fill_opacity=0.6,
    ).add_to(m)

m

### Marker Clusters

In [ ]:
m_cluster = folium.Map(location=[0, 0], zoom_start=2)

np.random.seed(42)
random_points = [[np.random.uniform(-60, 60), np.random.uniform(-150, 150)] for _ in range(100)]

marker_cluster = MarkerCluster().add_to(m_cluster)
for point in random_points:
    folium.Marker(location=point, popup=f"Point: {point[0]:.2f}, {point[1]:.2f}").add_to(marker_cluster)

m_cluster

### Heatmap

In [ ]:
m_heat = folium.Map(location=[40, -95], zoom_start=4)

heat_data = [[row["lat"], row["lon"], row["population"]] for idx, row in cities.iterrows()]
HeatMap(heat_data, radius=50).add_to(m_heat)

m_heat

### Multiple Tile Layers

In [ ]:
m_tiles = folium.Map(location=[40.7128, -74.0060], zoom_start=12)

folium.TileLayer("OpenStreetMap").add_to(m_tiles)
folium.TileLayer("cartodbpositron", name="CartoDB Positron").add_to(m_tiles)
folium.TileLayer("cartodbdark_matter", name="CartoDB Dark").add_to(m_tiles)
folium.LayerControl().add_to(m_tiles)

folium.Marker(
    location=[40.7128, -74.0060],
    popup="New York City",
    icon=folium.Icon(color="red", icon="info-sign"),
).add_to(m_tiles)

m_tiles

---

## Summary

In this notebook, we covered:

1. **Cartopy**: Map projections, coastlines, borders, and plotting data on maps
2. **GeoPandas**: GeoDataFrames, spatial operations (distance, centroid, buffer)
3. **Folium**: Interactive maps, markers, clusters, heatmaps, and tile layers

For more information:
- [Cartopy Gallery](https://scitools.org.uk/cartopy/docs/latest/gallery/)
- [GeoPandas Documentation](https://geopandas.org/en/stable/docs.html)
- [Folium Documentation](https://python-visualization.github.io/folium/)